In [199]:
import pandas as pd
import requests

from datetime import datetime

from scrapy import Selector

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [200]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

html = requests.get(url).text
sel = Selector(text = html)

nasdaq100_df = pd.DataFrame()

for n in range(2,104):
    ticker = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[2]'.format(n)).extract()[0].strip('</td>')
    company_name = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[1]/a/@title'.format(n)).get()
    industry = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[3]'.format(n)).get().strip('</td>')
    sub_industry = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[3]'.format(n)).get().strip('</td>')
    new_list = [ticker, company_name, industry, sub_industry]
    pd_series_row = pd.Series(new_list, index = ['ticker', 'company_name', 'industry', 'sub_industry'])
    nasdaq100_df = nasdaq100_df.append(pd_series_row, ignore_index=True)
    print(ticker, end = ' ')

nasdaq100_df.head(4)

ATVI ADBE AMD ALGN GOOGL GOOG AMZN AEP AMGN ADI ANSS AAPL AMAT ASML TEAM ADSK ADP BIDU BIIB BKNG AVGO CDNS CDW CERN CHTR CHKP CTAS CSCO CTSH CMCSA CPRT COST CRWD CSX DXCM DOCU DLTR EBAY EA EXC FAST FISV FOXA FOX GILD HON IDXX ILMN INCY INTC INTU ISRG JD KDP KLAC KHC LRCX LULU MAR MRVL MTCH MELI FB MCHP MU MSFT MRNA MDLZ MNST NTES NFLX NVDA NXPI ORLY OKTA PCAR PAYX PYPL PTON PEP PDD QCOM REGN ROST SGEN SIRI SWKS SPLK SBUX SNPS TMUS TSLA TXN TCOM VRSN VRSK VRTX WBA WDAY XEL XLNX ZM 

,company_name,industry,sub_industry,ticker
0,Activision Blizzard,Communication Services,Communication Services,ATVI
1,Adobe Inc.,Information Technology,Information Technology,ADBE
2,Advanced Micro Devices,Information Technology,Information Technology,AMD
3,Align Technology,Health Care,Health Care,ALGN


In [201]:
def unix_to_date(unix_timestamp):
    ts = int(unix_timestamp)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

In [202]:
with open('/Users/ilya/Desktop/keys/yahoo_finance.txt','r') as file:
    key = file.readlines()

KEY = key[0]

In [203]:
def get_weekly_history_df(symbol):
    get_history_url = "https://yh-finance.p.rapidapi.com/stock/v3/get-historical-data"

    querystring = {"symbol": symbol,"region":"US"}

    headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': KEY
    }

    response = requests.request("GET", get_history_url, headers=headers, params=querystring)

    json_response = response.json()

    weekly_history_df = pd.DataFrame(json_response['prices'])[['date','close']]
    weekly_history_df.loc[:,'date'] = weekly_history_df['date'].apply(lambda x: unix_to_date(x))

    weekly_history_df.loc[:,'week'] = [pd.Timestamp(date).week for date in weekly_history_df['date']]
    weekly_history_df.loc[:,'year'] = [pd.Timestamp(date).year for date in weekly_history_df['date']]

    weekly_history_df.loc[:,'close'] = weekly_history_df['close'].round(1)

    return weekly_history_df[['date','year','week','close']]

In [204]:
def get_revenue_and_earnings(ticker, period):
    financials_url = "https://yh-finance.p.rapidapi.com/stock/v2/get-financials"

    querystring = {"symbol":ticker,"region":"US"}

    headers = {
        'x-rapidapi-host': "yh-finance.p.rapidapi.com",
        'x-rapidapi-key': KEY
        }

    financials_response = requests.request("GET", financials_url, headers=headers, params=querystring).json()

    ##these are quarterly earnings specifically
    revenue_df = pd.DataFrame()
    earnings_df = pd.DataFrame()

    for quarter in financials_response['incomeStatementHistoryQuarterly']['incomeStatementHistory']:
        revenue_line = pd.Series([quarter['endDate']['fmt'], round(quarter['totalRevenue']['raw']/1000000000,1)],
        index = ['quarter','revenue_billion'])
        revenue_df = revenue_df.append(revenue_line, ignore_index=True)

    revenue_df['quarter'] = [pd.Timestamp(quarter) for quarter in revenue_df['quarter']]
    revenue_df['quarter'] = ['4Q2020' if quarter <= pd.Timestamp('2021-01-01T12') else
                             '1Q2021' if quarter > pd.Timestamp('2021-01-02T12') and quarter < pd.Timestamp('2021-04-01T12') else
                             '2Q2021' if quarter > pd.Timestamp('2021-04-02T12') and quarter < pd.Timestamp('2021-07-01T12') else
                             '3Q2021' if quarter > pd.Timestamp('2021-07-02T12') and quarter < pd.Timestamp('2021-10-02T12') else
                             '4Q2021' for quarter in revenue_df['quarter']]

    for quarter in financials_response['earnings']['earningsChart']['quarterly']:
        earning_line = pd.Series([quarter['date'], quarter['actual']['raw']], index = ['quarter','earnings_billion'])
        earnings_df = earnings_df.append(earning_line, ignore_index=True)

    revenue_earnings_df = revenue_df.merge(earnings_df, left_on = ['quarter'], right_on = ['quarter'])

    revenue_earnings_df['profitability'] = round(revenue_earnings_df['earnings_billion']/revenue_earnings_df['revenue_billion'],2)

    revenue_earnings_df['ticker'] = ticker

    return revenue_earnings_df.head(period)

In [205]:
def get_la_ratio(symbol, quarters):
  balance_sheet_url = "https://yh-finance.p.rapidapi.com/stock/v2/get-balance-sheet"

  querystring = {"symbol":symbol,"region":"US"}

  headers = {
            'x-rapidapi-host': "yh-finance.p.rapidapi.com",
            'x-rapidapi-key': KEY
            }

  response = requests.request("GET", balance_sheet_url, headers=headers, params=querystring)

  balance_sheet_response = response.json()

  balance_df = pd.DataFrame()

  for item in balance_sheet_response['balanceSheetHistoryQuarterly']['balanceSheetStatements']:
    balance_line = pd.Series([item['endDate']['fmt'],
                              item['totalCurrentLiabilities']['fmt'],
                              item['totalCurrentAssets']['fmt']],
                              #item['commonStock']['fmt']], --> some companies don't display this?
                              index = ['quarter','liabilities','assets'])

    balance_df = balance_df.append(balance_line, ignore_index = True)

  balance_df.loc[:,'assets_billion'] = balance_df['assets'].str.strip('B').astype('float')
  balance_df.loc[:,'liabilties_billion'] = balance_df['liabilities'].str.strip('B').astype('float')
  balance_df.loc[:,'la_ratio'] = round(balance_df['liabilties_billion']/balance_df['assets_billion'],2)
  balance_df = balance_df.drop(columns = ['assets','liabilities'])
  balance_df['ticker'] = symbol

  return balance_df.head(quarters)

In [206]:
def get_pe_ratio(symbol, quarters):
    stats_url = "https://yh-finance.p.rapidapi.com/stock/v2/get-statistics"

    querystring = {"symbol":symbol,"region":"US"}

    headers = {
              'x-rapidapi-host': "yh-finance.p.rapidapi.com",
              'x-rapidapi-key': "f31c8be57dmsh32b4005c3178b54p1f4272jsnb55bf64ce33e"
              }

    stats_response = requests.request("GET", stats_url, headers=headers, params=querystring).json()

    ratio_df = pd.DataFrame()

    for quarter in stats_response['timeSeries']['quarterlyPeRatio']:
        if quarter is not None:
            pd_row = pd.Series([quarter['asOfDate'], quarter['reportedValue']['fmt']],
            index = ['quarter','pe_ratio'])
            ratio_df = ratio_df.append(pd_row, ignore_index=True)

    ratio_df['ticker'] = symbol
        
    return ratio_df.sort_values(by = 'quarter', ascending = False).head(quarters)

In [207]:
def list_perfomance(symbol):
    weekly_history_df = get_weekly_history_df(symbol)

    performance_list = [symbol,\
                        round((weekly_history_df.iloc[0]['close'] - weekly_history_df.iloc[-1]['close'])/weekly_history_df.loc[0]['close'],3),\
                        round(weekly_history_df['close'].std(),2),
                        round(weekly_history_df['close'].mean(),2),
                        round(weekly_history_df['close'].std()/weekly_history_df['close'].mean(),2),
                        round(weekly_history_df.iloc[0]['close'],1)
                        ]

    return performance_list

In [209]:
counter = 0

attributes = ['ticker','yoy_growth','std','mean_price','volatility','last_close_price']
comparison_df = pd.DataFrame(columns = attributes)

for ticker_name in list(nasdaq100_df['ticker'][0:20]):
    series_row = pd.Series(list_perfomance(ticker_name), index = attributes)
    comparison_df = comparison_df.append(series_row, ignore_index=True)
    print(ticker_name, end = ' ')

la_ratio_main_df = pd.DataFrame()

for ticker_name in list(nasdaq100_df['ticker'][0:20]):
    mini_df = get_la_ratio(ticker_name,1)
    la_ratio_main_df = la_ratio_main_df.append(mini_df)
    print(ticker_name, end = ' ')

revenue_earnings_main_df = pd.DataFrame()

for ticker_name in list(nasdaq100_df['ticker'][0:20]):
    mini_df = get_revenue_and_earnings(ticker_name,1)
    revenue_earnings_main_df = revenue_earnings_main_df.append(mini_df)
    print(ticker_name, end = ' ')

pe_ratio_main_df = pd.DataFrame()

for ticker_name in list(nasdaq100_df['ticker'][0:20]):
    mini_df = get_pe_ratio(ticker_name,1)
    pe_ratio_main_df = pe_ratio_main_df.append(mini_df)
    print(ticker_name, end = ' ')

ATVI ADBE AMD ALGN GOOGL GOOG AMZN AEP AMGN ADI ANSS AAPL AMAT ASML TEAM ADSK ADP BIDU BIIB 

In [197]:
final_df = comparison_df.merge(la_ratio_main_df, left_on = ['ticker'], right_on = ['ticker'])\
                        .merge(revenue_earnings_main_df, left_on = ['ticker'], right_on = ['ticker'])\
                        .merge(pe_ratio_main_df, left_on = ['ticker'], right_on = ['ticker'])

final_df = final_df.merge(nasdaq100_df, left_on = ['ticker'], right_on = ['ticker'])

final_df = final_df[['ticker','company_name','industry','sub_industry','last_close_price','yoy_growth','volatility',
                    'assets_billion','liabilties_billion','la_ratio',
                    'revenue_billion','earnings_billion','profitability',
                    'pe_ratio']]

In [198]:
final_df

,ticker,company_name,industry,sub_industry,last_close_price,yoy_growth,volatility,assets_billion,liabilties_billion,la_ratio,revenue_billion,earnings_billion,profitability,pe_ratio
0,MU,Micron Technology,Information Technology,Information Technology,83.400,0.230,0.090,19.910,6.420,0.320,8.300,2.420,0.290,20.36
1,MSFT,Microsoft,Information Technology,Information Technology,329.700,0.347,0.130,174.330,80.530,0.460,45.300,2.270,0.050,35.02
2,MRNA,Moderna,Health Care,Health Care,329.600,0.615,0.440,13.430,9.960,0.740,5.000,7.700,1.540,48.11
